In [35]:
# %pip install retina-face deepface
%pip install git+https://github.com/serengil/retinaface.git

  Cloning https://github.com/serengil/retinaface.git to /tmp/pip-req-build-4j59k0nq
  Running command git clone --filter=blob:none --quiet https://github.com/serengil/retinaface.git /tmp/pip-req-build-4j59k0nq
  Resolved https://github.com/serengil/retinaface.git to commit 3ddbbd2e6f3b0e49b51b9ba76a7ad03af4486c5f
  Preparing metadata (setup.py) ... done
  Created wheel for retina-face: filename=retina_face-0.0.15-py3-none-any.whl size=24236 sha256=76c4d463637f7f72065e11d99a0e00988adefdd5ea4fd3c669f2f10550b5293f
  Stored in directory: /tmp/pip-ephem-wheel-cache-76z0wn0a/wheels/16/27/32/e83a80b48c2100157aff36ae7cbda0160468770403faf90027
Successfully built retina-face
  Attempting uninstall: retina-face
    Found existing installation: retina-face 0.0.13
    Uninstalling retina-face-0.0.13:
      Successfully uninstalled retina-face-0.0.13

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel

In [2]:
from retinaface import RetinaFace
help(RetinaFace.extract_faces)

Help on function extract_faces in module retinaface.RetinaFace:

extract_faces(img_path: Union[str, numpy.ndarray], threshold: float = 0.9, model: Optional[keras.engine.training.Model] = None, align: bool = True, allow_upscaling: bool = True, expand_face_area: int = 0, align_first: bool = False) -> list
    Extract detected and aligned faces
    Args:
        img_path (str or numpy): given image
        threshold (float): detection threshold
        model (Model): pre-trained model can be passed to the function
        align (bool): enable or disable alignment
        allow_upscaling (bool): allowing up-scaling
        expand_face_area (int): expand detected facial area with a percentage
        align_first (bool): set this True to align first and detect second
            this can be applied only if input image has just one face



In [6]:
import os

import cv2
import numpy as np
from retinaface import RetinaFace

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model = RetinaFace.build_model()


def detect_and_save_faces(input_image_path, output_folder, face_idx):
    # Load the input image
    image = cv2.imread(input_image_path)

    try:
        # Detect faces using the SSD model
        detected_faces = RetinaFace.extract_faces(
            input_image_path,
            threshold=0.97,
            model=model,
            align=True,
            align_first=True,
            allow_upscaling=True,
            expand_face_area=10,
        )
        
        if len(detected_faces) > 1:
            print(f'{input_image_path} has more than 1 face.')
            return face_idx

        # Create the output folder if it doesn't exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Loop through each detected face and save it as a 300x300 PNG file
        for i, face_info in enumerate(detected_faces):
            face = face_info
            # print(face)

            print("resizing..")
            maxwidth, maxheight = 300, 300
            f1 = maxwidth / face.shape[1]
            f2 = maxheight / face.shape[0]
            f = min(f1, f2)
            dim = (int(face.shape[1] * f), int(face.shape[0] * f))
            face_resized = cv2.resize(face, dim, interpolation=cv2.INTER_AREA)

            face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)

            # Create a black image with the target size
            padded_image = np.zeros((300, 300, 3), dtype=np.uint8)
            padding_rows = (300 - dim[1]) // 2
            padding_cols = (300 - dim[0]) // 2

            # Add the resized image to the padded image, with padding on the left and right sides
            padded_image[padding_rows : padding_rows + dim[1], padding_cols : padding_cols + dim[0]] = face_rgb

            face_idx += 1
            # Save the face as a PNG file in the output folder
            output_path = os.path.join(output_folder, f"face_{face_idx}.png")
            cv2.imwrite(output_path, padded_image)

        print(f"{len(detected_faces)} faces detected and saved to {output_folder}")
    except Exception as e:
        print(str(e))

    return face_idx


import os

path_img = "images"
path_faces = "faces"

# # test
# input_image_path = '01.png'
# output_folder = 'output/'
# detect_and_save_faces(input_image_path, output_folder, 0)

for gender in sorted(os.listdir(path_img)):
    path_gender = os.path.join(path_img, gender)
    for name in sorted(os.listdir(path_gender)):
        if os.path.exists(os.path.join(path_faces, gender, name)):
            continue
        
        path_name = os.path.join(path_img, gender, name)
        face_idx = 0
        for path_file in sorted(os.listdir(path_name)):
            path_img_file = os.path.join(path_name, path_file)
            if os.path.getsize(path_img_file) < 200000:
                continue
            print(path_img_file)
            # input_image_path = '01.png'
            output_folder = os.path.join(path_faces, gender, name)
            face_idx = detect_and_save_faces(path_img_file, output_folder, face_idx)


images/남자/BTS RM/03.png
resizing..
1 faces detected and saved to faces/남자/BTS RM
images/남자/BTS RM/09.png
resizing..
1 faces detected and saved to faces/남자/BTS RM
images/남자/BTS RM/10.png
resizing..
1 faces detected and saved to faces/남자/BTS RM
images/남자/BTS RM/108.png
24-02-13 09:03:42 - ⚠️ Even though align_first is set to True, there are 3 faces in input image.Align first functionality can be applied only if there is single face in the input
images/남자/BTS RM/108.png has more than 1 face.
images/남자/BTS RM/112.png
resizing..
1 faces detected and saved to faces/남자/BTS RM
images/남자/BTS RM/115.png
resizing..
1 faces detected and saved to faces/남자/BTS RM
images/남자/BTS RM/116.png
24-02-13 09:03:56 - ⚠️ Even though align_first is set to True, there are 2 faces in input image.Align first functionality can be applied only if there is single face in the input
images/남자/BTS RM/116.png has more than 1 face.
images/남자/BTS RM/119.png
24-02-13 09:04:00 - ⚠

KeyboardInterrupt: 